In [1]:
from utils import parse_args
from DataModule import DataModule
from models.JEM import get_model, get_optim
from accelerate.utils import set_seed
import torch as t
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

Please install the required packages first. Use `pip install -r requirements.txt`.


ImportError: cannot import name 'issparse' from 'scipy.sparse' (unknown location)

In [ ]:
args = parse_args()
set_seed(args.seed)

usage: ipykernel_launcher.py [-h] [--model MODEL] [--lr LR] [--optim OPTIM]
                             [--norm {none,batch}]
                             [--decay_epochs DECAY_EPOCHS [DECAY_EPOCHS ...]]
                             [--px PX] [--pyx PYX] [--l2 L2]
                             [--n_steps N_STEPS] [--in_steps IN_STEPS]
                             [--query_size QUERY_SIZE]
                             [--sample_method SAMPLE_METHOD] [--test]
                             [--temp_scale] [--dataset DATASET]
                             [--width WIDTH] [--depth DEPTH]
                             [--decay_rate DECAY_RATE] [--n_epochs N_EPOCHS]
                             [--batch_size BATCH_SIZE] [--clf_only CLF_ONLY]
                             [--warmup_iters WARMUP_ITERS]
                             [--dropout_rate DROPOUT_RATE] [--sigma SIGMA]
                             [--weight_decay WEIGHT_DECAY]
                             [--buffer_size BUFFER_SIZE]
         

SystemExit: 2

/home/lorenzoquerol/miniconda3/envs/JEMPP/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
datamodule = DataModule()
dload_train, dload_train_labeled, dload_train_unlabeled, dload_valid, train_labeled_inds, train_unlabeled_inds = datamodule.get_data()

f = get_model(datamodule, args)

ckpt = t.load("/home/lorenzo/repos/JEMPP/runs/2023-12-06_21-11-29_bloodmnist_l760ez83/checkpoints/baseline_6400/epoch=77-val_loss=0.1389.ckpt")
f.load_state_dict(ckpt["model_state_dict"])

# generate features
features = []
labels = []
device = "cuda" if t.cuda.is_available() else "cpu"

f = f.to(device)
for i, (x, y) in enumerate(tqdm(dload_train, desc="Generating features")):
    x, y = x.to(device), y.to(device)
    features.append(f.feature(x).detach().cpu())
    labels.append(y.detach().cpu())

features = t.cat(features, dim=0)
labels = t.cat(labels, dim=0)

tsne = TSNE(random_state=args.seed)
tsne_output = tsne.fit_transform(features)

df = pd.DataFrame(tsne_output, columns=["x", "y"])
df["targets"] = labels
df["targets"] = df["targets"].apply(lambda x: datamodule.classes[x])

plt.rcParams["figure.figsize"] = 10, 10
scatter_plot = plt.scatter(
    x="x",
    y="y",
    marker="o",
    alpha=0.75,
)
scatter_plot.legend(bbox_to_anchor=(1.05, 1), loc="upper right")

plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

plt.show()